In [2]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

import numpy as np
import pandas as pd 
import json

import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import BertTokenizer, get_linear_schedule_with_warmup

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import matthews_corrcoef, confusion_matrix

import random
import wandb

import sys
sys.path.append('/home/inaki/host_data')
import models
from utils import train_loop, evaluate, evaluate_kfold_ensemble
from CreateDataset import BertDataset

In [3]:
###############################
#       configuración         #
###############################
MODEL_NAME = "BertBaseDenseLogits"

MAX_LENGTH = 512
HEAD_DROPOUT = 0.0
DATA_AUGMENTATION = [] #["Oversampling"] # '_es_into_en' # 'llama_aug'
FOLDS_NUM = 10

NUM_EPOCHS = 10
LEARNING_RATE = 2e-5
BATCH_SIZE = 16
DECAY = 0

LOSS_FN =  'supervised_contrastive' # 'cross_entropy' # 'supervised_contrastive'

temperature = 0.3  # temprature for contrastive loss
lam = 0.9  # lambda for loss

checkoint_folder = 'Baseline'

datadir = "/home/inaki/host_data/dataset_oppositional/"
cuda_device = 0
wandb_project = 'epoch_analysis'    # 'BASELINE'  'Ensemble_baseline'  'trash'  'epoch_analysis'

In [4]:
!nvidia-smi

Fri May 10 17:23:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:18:00.0 Off |                  N/A |
| 30%   35C    P8              29W / 350W |    780MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# Cargamos los datasets de entrenamiento y test
train_en_dataset_path = datadir + "train_en_data.pth"
X, y = torch.load(train_en_dataset_path)
X = np.array(X)
y = np.array(y)
train_en_dataset = BertDataset(X, y)

# Obtén la referencia a la clase del módulo models
ModelClass = getattr(models, MODEL_NAME)
model = ModelClass(dropout_prob=HEAD_DROPOUT, dense_dim=32)

device = torch.device(f"cuda:{cuda_device}" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
for param in model.parameters():
    param.requires_grad = True

cuda:0


In [11]:

temp = 0.3
dataloader = DataLoader(train_en_dataset, batch_size=5, shuffle=False)
input_ids, attention_mask, labels, _ = dataloader.__iter__().__next__()
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

embedding, logit = model(input_ids, attention_mask)

norm_L2 = torch.norm(embedding, p=2, dim=1, keepdim=True)
print(norm_L2)

# cosine similarity between embeddings
cosine_sim = torch.nn.functional.cosine_similarity(embedding.unsqueeze(1), embedding.unsqueeze(0), dim=2)
print(cosine_sim)


#print(dis)

# apply temperature to elements
dis = dis / temp
#print(dis)
cosine_sim = cosine_sim / temp

# apply exp to elements
dis = torch.exp(dis)
print(dis)
cosine_sim = torch.exp(cosine_sim)
# calculate row sum
row_sum = dis.sum(dim=1)
print(row_sum)

# calculate outer sum
contrastive_loss = 0
print(labels)
for i in range(len(embedding)):
    n_i = (labels == labels[i]).sum().item() - 1
    print(n_i)
    inner_sum = 0
"""
    # calculate inner sum
    for j in range(len(embedding)):
        if label[i] == label[j] and i != j:
            inner_sum = inner_sum + torch.log(cosine_sim[i][j] / row_sum[i])
    if n_i != 0:
        contrastive_loss += (inner_sum / (-n_i))
    else:
        contrastive_loss += 0
"""

tensor([[19.1925],
        [18.4902],
        [15.5536],
        [18.8394],
        [19.6545]], device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)
tensor([[1.0000, 0.9836, 0.9393, 0.9550, 0.9926],
        [0.9836, 1.0000, 0.9403, 0.9510, 0.9715],
        [0.9393, 0.9403, 1.0000, 0.9334, 0.9195],
        [0.9550, 0.9510, 0.9334, 1.0000, 0.9498],
        [0.9926, 0.9715, 0.9195, 0.9498, 1.0000]], device='cuda:0',
       grad_fn=<SumBackward1>)
tensor([[ 1.0000, 26.5409, 22.9002, 24.1306, 27.3483],
        [26.5409,  1.0000, 22.9765, 23.8062, 25.4928],
        [22.9002, 22.9765,  1.0000, 22.4483, 21.4311],
        [24.1306, 23.8062, 22.4483,  1.0000, 23.7140],
        [27.3483, 25.4928, 21.4311, 23.7140,  1.0000]], device='cuda:0',
       grad_fn=<ExpBackward0>)
tensor([101.9200,  99.8164,  90.7561,  95.0991,  98.9863], device='cuda:0',
       grad_fn=<SumBackward1>)
tensor([0, 1, 0, 0, 0], device='cuda:0')
3
0
3
3
3


'\n    # calculate inner sum\n    for j in range(len(embedding)):\n        if label[i] == label[j] and i != j:\n            inner_sum = inner_sum + torch.log(cosine_sim[i][j] / row_sum[i])\n    if n_i != 0:\n        contrastive_loss += (inner_sum / (-n_i))\n    else:\n        contrastive_loss += 0\n'